In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime   # date stamp the log directory
import json  # for saving and loading hyperparameters
import os, sys, re
import time

import absl
import absl.logging as logging
from tf2_models.matrix_caps import *

gfile = tf.io.gfile
flags = absl.app.flags

In [3]:
class CapsConfig(object):
  def __init__(self,
               **kwargs):
    self.output_dim = 10
    self.A = 64
    self.B = 8
    self.C = 16
    self.D = 16
    self.epsilon = 1e-9
    self.l2 = 0.0000002
    self.final_lambda = 0.01
    self.iter_routing = 3

In [ ]:
import tensorflow_datasets as tfds
orig = tfds.load('smallnorb', split="train")
dataset = orig.map(map_func=lambda x: (tf.cast(x['image'], tf.float32), tf.one_hot(x['label_category'],depth=10)),
                                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(2)
x,y = next(iter(dataset))

In [ ]:
valid_dataset = tfds.load('smallnorb', split="train")
valid_dataset = valid_dataset.map(map_func=lambda x: (tf.cast(x['image'], tf.float32), tf.one_hot(x['label_category'],depth=10)),
                                              num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(2)

In [ ]:
model = MatrixCaps(CapsConfig())
outputs = model(x)
print(outputs.shape)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=tf.keras.metrics.SparseCategoricalAccuracy())

model.summary()

In [ ]:
outputs = model(x)
tf.print(outputs.shape)
tf.print(y.shape)
model.loss(y, outputs)

In [ ]:
model.fit(x=x.numpy(), y=y.numpy(),
          epochs=1,
          verbose=2
          )

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are Numpy arrays)
#x_train = x_train.reshape(60000, 784).astype('float32') / 255
#x_test = x_test.reshape(10000, 784).astype('float32') / 255

x_train = x_train[...,None].astype('float32')
x_test = x_test[...,None].astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]
tf.print(x_train.shape)

(50000, 28, 28, 1)


In [5]:
model = MatrixCaps(CapsConfig())
outputs = model(x_train[0:2])
print(outputs.shape)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

(2, 10)
Model: "matrix_caps"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d (Conv2D)              multiple                  1664      
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  8320      
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  520       
_________________________________________________________________
conv_caps1 (ConvCaps)        multiple                  18464     
_________________________________________________________________
conv_caps2 (ConvCaps)        multiple                  36896     
_________________________________________________________________
class_caps (FcCaps)          multiple          

In [6]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 1584s 32ms/sample - loss: 102.4443 - sparse_categorical_accuracy: 0.0933 - val_loss: 103.1420 - val_sparse_categorical_accuracy: 0.0960
Epoch 2/3
10816/50000 [=====>........................] - ETA: 19:00 - loss: 102.4235 - sparse_categorical_accuracy: 0.0912

KeyboardInterrupt: 

In [ ]:
inputs = tf.keras.Input(shape=(28,28,1), name='digits')
x = tf.keras.layers.Reshape(target_shape=(784,))(inputs)
x = tf.keras.layers.Dense(64, activation='relu', name='dense_1')(x)
x = tf.keras.layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = tf.keras.layers.Dense(10, name='predictions')(x)

model2 = tf.keras.Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
out = model2(x_train[0:1])
print(out.shape)
history = model2.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

In [ ]:
model = MatrixCaps(CapsConfig())
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
print(x_train[0:4].shape)
out = model(x_train[0:4])
print(out.shape)
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    validation_data=(x_val, y_val))

print('\nhistory dict:', history.history)

In [ ]:
tf.tile(tf.eye(10)[5][None,...],(3,1))